# Librairies

In [1]:
import pandas as pd 
import numpy as np

# 1 - Les données

In [2]:
data= pd.read_csv("../Data/Data_marketing_campaign.csv", sep ="\t")

In [3]:
data.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


In [4]:
rows, columns = data.shape
print(f"La base de données contient {rows} lignes et {columns} colonnes.")

La base de données contient 2240 lignes et 29 colonnes.


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

In [6]:
# Vérification de l'unicité des ID
duplicates = data[data['ID'].duplicated()].sum().sum()
print(int(duplicates))

0


>>Aucune donnée client n'est dupliquée. Toutes les lignes contiennent des identifiants de clients uniques dans les données.

In [7]:
missing_vars = data.isna().sum()

print("Variables avec valeurs manquantes et leur proportion :")
print("\n".join([f"- {var} : {count} ({count / data.shape[0] * 100:.2f}%)" 
                 for var, count in missing_vars[missing_vars > 0].items()]))


Variables avec valeurs manquantes et leur proportion :
- Income : 24 (1.07%)


>>Il y a 24 clients dont le revenu n'est pas renseigné, représentant 1,07 % des données. Étant donné la faible proportion, le choix est fait de les exclure de l'analyse.

In [8]:
df = data.copy()
df = df.dropna()

# 2 Recodage des modalités de certains attributs des clients.

## 2.1 Statut marital du client

In [9]:
print(f"""Les modalités de Marital_Status sont : {", ".join(df["Marital_Status"].unique())}""")

Les modalités de Marital_Status sont : Single, Together, Married, Divorced, Widow, Alone, Absurd, YOLO


>>Les modalités `Single` et `Together` sont choisies pour simplifier l'analyse des statuts maritaux. Les catégories suivantes sont regroupées sous `Single` parce qu'elles ne représentent pas une situation où le client est en couple :`Divorced`, `Widow`, `Alone`, `Absurd` et `YOLO` <br> <br> De même, les modalités `Married` et `Together` sont fusionnées sous `Together` pour refléter les personnes engagées dans une relation, qu'elle soit formelle ou informelle. Ce recodage permet une meilleure clarté et une analyse plus simple.


In [10]:
df['Marital_Status'] = df['Marital_Status'].replace({
                                                    'YOLO': 'Single',
                                                    'Alone': 'Single',
                                                    'Absurd': 'Single',
                                                    'Widow': 'Single',
                                                    'Divorced': 'Single',
                                                    'Married': 'Together'
                                                })


In [11]:
print (f"""Les nouvelles modalités de Marital_satus sont : {" et ".join(df["Marital_Status"].unique())}""")

Les nouvelles modalités de Marital_satus sont : Single et Together


### 2.1.1 Niveau d'éducation du client

In [12]:
print(f"""Les valeurs initiales du niveau d'éducation des clients sont { ', '.join(df["Education"].unique())}""")

Les valeurs initiales du niveau d'éducation des clients sont Graduation, PhD, Master, Basic, 2n Cycle


``Graduation`` sera considérée comme la modalité centrale. Tous les niveaux d'étude inférieurs seront remplacés par ``Undergraduate`` (``2n Cycle`` et ``Basic``), tandis que les niveaux supérieurs seront classés comme ``Postgraduate`` (``PhD`` et ``Master``).


In [13]:
df["Education"] = df["Education"].replace({"PhD" : "PostGraduation",
                                              "Master" : "PostGraduation",
                                              "2n Cycle" : "UnderGraduation",
                                              "Basic" : "UnderGraduation"
                                              })

In [14]:
print(f"""Les valeurs finales du niveau d'éducation des clients sont { ", ".join(df["Education"].unique())}""")

Les valeurs finales du niveau d'éducation des clients sont Graduation, PostGraduation, UnderGraduation


# 3 Création de variables

Ces variables seront créées et définies ainsi :

- **"Is_parent"** : prend la valeur `0` si le nombre d'enfants et de jeunes est égal à 0, et `1` dans le cas contraire.
- **"Household"** : nombre de personnes dans le ménage.
- **"Age"** : âge du client en 2014 (année de la collecte des données), calculé à partir de sa date de naissance `Year_Birth`.
- **"Seniority"** : ancienneté du client en 2014 (année de la collecte des données), calculée à partir de la date d’inscription `Dt_Customer`.
- **"Total_Spent"** : montant total dépensé sur l'ensemble des produits achetés.
- **"Total_Accepted_Campaigns"** : total de campagne accepté par le client.


In [15]:
# Is_parent : Le client est-il un parent ?

df['Is_parent'] = np.where((df['Kidhome'] >= 1) | (df['Teenhome'] >= 1), "Parent", "Not Parent")

In [16]:
# Household : combien de personnes (enfants + clients) vivent dans le ménage ? 
# On ajoute une personne supplémentaire si le client a déclaré être en couple (Together)

df['Household'] = np.where(df["Marital_Status"]=="Together", 2 + df['Kidhome'] + df['Teenhome'], 1 + df['Kidhome'] + df['Teenhome'])

In [17]:
# age : quel est l'age du client ?

df['Age'] = 2014 - df.Year_Birth

In [18]:
# Seniority : quelle est l'ancienneté du client ?

df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], dayfirst=True)
df['Seniority'] = (df['Dt_Customer'].max() - df['Dt_Customer']).dt.days

In [19]:
# Total_Spent : combien le client a-il dépensé au total ?

df['Total_Spent'] = df[['MntWines' , 'MntFruits' , 'MntMeatProducts' , 'MntFishProducts', 'MntSweetProducts' , 'MntGoldProds']].sum(axis=1)

In [20]:
# Total_Accepted_Campaigns : combien de campagnes le client a-t-il accepté ?

df['Total_Accepted_Campaigns'] = df[['AcceptedCmp3','AcceptedCmp4',  'AcceptedCmp5', 'AcceptedCmp1','AcceptedCmp2']].sum(axis=1)


In [21]:
# Total_purchases : nombre total d'achats réalisés ?

df["Total_purchases"] = df[["NumDealsPurchases", "NumWebPurchases", "NumCatalogPurchases", "NumStorePurchases"]].sum(axis=1)

# 4 Supression de variables redondantes et qui ne seront pas utilisés pour la modélisation et export de la donnée

In [22]:
to_drop = ['AcceptedCmp3','AcceptedCmp4','AcceptedCmp5','AcceptedCmp1','AcceptedCmp2','Kidhome', 'Teenhome','Z_CostContact','Z_Revenue', 'Dt_Customer', 'Year_Birth', "ID"]
df = df.drop(columns=to_drop)

In [23]:
df.columns

Index(['Education', 'Marital_Status', 'Income', 'Recency', 'MntWines',
       'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'Complain', 'Response', 'Is_parent', 'Household', 'Age', 'Seniority',
       'Total_Spent', 'Total_Accepted_Campaigns', 'Total_purchases'],
      dtype='object')

In [24]:
df.to_csv("../Data/Data_vi_marketing_campaigns.csv", index=False)